In [1]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [2]:
#1. build a random forest classifier with 'longer' text
import pandas as pd
df_train = pd.read_csv("/Users/chengyiyang/Desktop/tweets_fltrd_4 - Copy.csv", engine='python')
df_train

,Unnamed: 0,Unnamed: 0.1,Tokens,Annotation,lang,Label
0,0,0,muslim mob violence against hindus in banglade...,racism,en,1
1,1,1,islamophobia is like the idea of naziphobia is...,racism,en,1
2,2,2,finally all caught up and that sudden death co...,none,en,0
3,4,4,please please start using is your discernment ...,none,en,0
4,5,5,as soon as isis chased all the minorities out ...,none,en,0
...,...,...,...,...,...,...
48358,53060,53060,give up jamal jrk off would never possibly was...,abuse,en,1
48359,53061,53061,soon we will all be pledging allegiance to the...,none,en,0
48360,53062,53062,never really gave it much thought just figured...,none,en,0
48361,53065,53065,leave your email or phone number and maybe you...,abuse,en,1


In [12]:
sub_df_train = df_train[['Tokens','Label']].copy()
sub_df_train

,Tokens,Label
0,muslim mob violence against hindus in banglade...,1
1,islamophobia is like the idea of naziphobia is...,1
2,finally all caught up and that sudden death co...,0
3,please please start using is your discernment ...,0
4,as soon as isis chased all the minorities out ...,0
...,...,...
48358,give up jamal jrk off would never possibly was...,1
48359,soon we will all be pledging allegiance to the...,0
48360,never really gave it much thought just figured...,0
48361,leave your email or phone number and maybe you...,1


In [22]:
from sklearn.model_selection import train_test_split
xs = sub_df_train['Tokens'].values
ys = sub_df_train['Label'].values
train_x, test_x, train_y, test_y = train_test_split(xs, ys, random_state=42, test_size=0.15)

In [23]:
print(len(train_x))
print(len(train_y))
print(len(test_x))
print(len(test_y
         ))


41108
41108
7255
7255


In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer

from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import TfidfTransformer 

vectorizer=CountVectorizer(max_features=100)


clf = RandomForestClassifier()
c = make_pipeline(vectorizer, clf)

c.fit(train_x, train_y)


TypeError: __init__() got an unexpected keyword argument 'stopwords'

In [42]:
vect_2 = CountVectorizer(stop_words='english', max_features=100)
c2 = make_pipeline(vect_2, clf)
c2.fit(train_x, train_y)


Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=100, stop_words='english')),
                ('randomforestclassifier', RandomForestClassifier())])

In [27]:
print (c.predict_proba(test_x[0:10]))

[[0.02538462 0.97461538]
 [0.64       0.36      ]
 [0.9284341  0.0715659 ]
 [1.         0.        ]
 [0.9295754  0.0704246 ]
 [0.95       0.05      ]
 [0.99       0.01      ]
 [0.83509756 0.16490244]
 [0.44       0.56      ]
 [0.92       0.08      ]]


In [28]:
print(test_x[0])
print(test_y[0])

yes only islam can generate that level of barbarity hatred and inhumanity
1


In [29]:
###############LIME#################

In [30]:
conda install -c conda-forge lime


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/chengyiyang/opt/anaconda3

  added / updated specs:
    - lime


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38h50d1736_0         3.1 MB  conda-forge
    lime-0.2.0.1               |     pyh9f0ad1d_0         233 KB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be INSTALLED:

  lime               conda-forge/noarch::lime-0.2.0.1-pyh9f0ad1d_0
  python_abi         conda-forge/osx-64::python_abi-

In [36]:
from lime.lime_text import LimeTextExplainer
class_names = ['Non-Toxic', 'Toxic']
explainer = LimeTextExplainer(class_names=class_names)

In [51]:
idx = 6666
exp = explainer.explain_instance(test_x[idx], c2.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(toxic) =', c2.predict_proba([test_x[idx]])[0:2,1])
print('True class: %s' % class_names[test_y[idx]])

Document id: 6666
Probability(toxic) = [0.82737079]
True class: Toxic


In [50]:
print(test_x[6666
            ])

islam is full of incitements to murder unbelievers what are you talking about


In [44]:
exp.as_list()

[('islam', 0.5901689300307933),
 ('talking', 0.19776764897573562),
 ('to', -0.00574572126096622),
 ('about', -0.005259497404806221),
 ('incitements', -0.004638497858986239),
 ('murder', -0.0038353184536634418)]